In [1]:
import json, os
import folium
import numpy as np
import seaborn as sns
import geopandas as gpd
from folium import plugins
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import gaussian_kde

# Introduction into handling geographical data

[John Snow](https://en.wikipedia.org/wiki/John_Snow) was an English physician, who was first to trace the source of a cholera outbreak in London's Soho district in 1854 using data visualization. In the following exercises, we are going to reconstruct the map of Snow with modern tools and explore some ideas of geographic data analysis.

<img src="https://upload.wikimedia.org/wikipedia/commons/2/27/Snow-cholera-map-1.jpg" width="400px"></img>

Though there is a whole [R library](https://vincentarelbundock.github.io/Rdatasets/doc/HistData/Snow.pumps.html) dedicated to this dataset, we would like R users to also build the solutions (e.g. Voronoi cells) from scratch.

Further reading:
* https://en.wikipedia.org/wiki/1854_Broad_Street_cholera_outbreak
* https://www1.udel.edu/johnmack/frec682/cholera/cholera2.html
* https://www.theguardian.com/news/datablog/interactive/2013/mar/15/cholera-map-john-snow-recreated

The shapefile that will be used in these exercises are located at http://donboyes.com/2011/10/14/john-snow-and-serendipity/, but they are already downloaded into the `/v/courses/dataesp2023.public/Datasets/D-Shapes-Maps-Coordinates/shapes` directory.

In [2]:
dataset_dir="/v/courses/2025-202425adatexplorcisvizualizci.public/Datasets/"

## Exercise 1

Load the death and the pump dataset into a tabular format! Have a look at the dataset's structure! The pump dataset contains the locations of wells. (Wells were used to obtain water for drinking, cooking etc. )

*The format of the files is called [shapefile](https://en.wikipedia.org/wiki/Shapefile), which is a format that is easily readable by the most common GIS (Geographic Information System) softwares. Shapefiles consist of many files, some of which store the geographic information, some the coordinate system description, some the additional attributes of the geographical objects. [geopandas](http://geopandas.org/) is able to read shapefiles natively into its base class GeoDataFrame, which has a similar API to that of `pandas` `DataFrames`. This is convenient, because we can use all of our previous knowledge of `pandas`.*

It is sometimes worth to have a look at a shapefile's content quickly. For Linux, [QGIS](https://www.qgis.org/en/site/about/index.html) is a powerful free GIS system, try it at home!

In [3]:
deaths = gpd.read_file(dataset_dir+"Shapes-Maps-Coordinates/shapes/deaths_gcs.shp")

In [4]:
deaths

,OBJECTID,X,Y,geometry
0,1,13.588010,11.095600,POINT (-0.13585 51.51295)
1,2,9.878124,12.559180,POINT (-0.13877 51.51371)
2,3,14.653980,10.180440,POINT (-0.13502 51.51249)
3,4,15.220570,9.993003,POINT (-0.13457 51.51239)
4,5,13.162650,12.963190,POINT (-0.13617 51.51387)
...,...,...,...,...
573,574,12.438040,11.536670,POINT (-0.13676 51.51318)
574,575,15.050640,10.174120,POINT (-0.13470 51.51248)
575,576,17.271660,11.633800,POINT (-0.13294 51.51317)
576,577,12.426100,11.914420,POINT (-0.13676 51.51336)


In [5]:
brpump = gpd.read_file(basedir+"/shapes/broad_st_pump_gcs.shp")

In [6]:
brpump

,OBJECTID,X,Y,geometry
0,1,12.57136,11.72717,POINT (-0.13664 51.51330)


In [7]:
pump = gpd.read_file(basedir+"/shapes/pumps_gcs.shp")

In [8]:
pump

,OBJECTID,X,Y,geometry
0,1,8.651201,17.891600,POINT (-0.13962 51.51630)
1,2,10.984780,18.517851,POINT (-0.13773 51.51658)
2,3,13.378190,17.394541,POINT (-0.13585 51.51602)
3,4,14.879830,17.809919,POINT (-0.13463 51.51620)
4,5,8.694768,14.905470,POINT (-0.13966 51.51487)
5,6,8.864416,12.753540,POINT (-0.13959 51.51383)
6,7,12.571360,11.727170,POINT (-0.13664 51.51330)
7,8,10.660970,7.428647,POINT (-0.13829 51.51125)
8,9,13.521460,7.958250,POINT (-0.13598 51.51148)
9,10,16.434891,9.252130,POINT (-0.13361 51.51207)


In [10]:
pump.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Exercise 2

Extract the latitudes and the longitudes for each row in both datasets as additional columns!

*In a `GeoDataFrame`, the `geometry` column contains geometry objects from the [`shapely`](https://toblerity.org/shapely/index.html) library. These objects have coordinate attributes, that is where latitude and longitude values are stored.*

The 'X' and 'Y' columns are coordinates in an arbitrary coordinate system and usually not part of a *shp* file. 

The "crs()" method tells us in what coordinate system are the coordinates. Make sure that you use the right coordinate system, in which you can measure distances! If needed, project your points into a coordinate system where you can measure real distances (using e.g. `pyproj`), then create a meaningful visualization of the data!

*It is crucial to use the appropriate coordinate system for your purposes. For this exercise, the spherical coordinate system of lon, lat values is not convenient, since we cannot calculate distances in meters as easily as we would in a Cartesian coordinate system. Because of the Earth's shape, most projections distort distances at certain latitudes and longitudes.*

In GIS, different coordinate systems are called CRS (Coordinate Reference System), and have patented codes that define the origin, the projection etc. The usual lon, lat pairs are called the WGS84 system (you will need to find its epsg code first). A good choice for a Cartesian system is for example the Pseudo-Mercator projection ([epsg=3857](https://epsg.io/3857)). The `pyproj` library is especially lightweight to use in Python.

## Exercise 3

How many deaths were within a given radius around the wells? Test for a series of radiuses. 

## Exercise 4

Put both the deaths and the pumps on a zoomable map that has a base layer from a tile server different from that of the default settings of your chosen method. Set the appropriate map center and zoom level. Use different markers for the deaths and the pumps datasets.

*Try folium in Python, or Leaflet in R or JS. Display the map here in the notebook.*

## Exercise 5

Calculate the Voronoi cells for the pumps, and count the number of deaths falling into each of the cells.  Create a bar chart of the number of deaths in each cell. Pinpoint the possible source of the infection.

*Store the Voronoi cells in a GeoDataFrame, then you can easily do a spatial join (e.g. testing whether a polygon in the Voronoi dataframe contains a point from the death dataframe).*

## Exercise 6

Create a death density map using 2D Kernel Density Estimation. Display the density and its contours on the map along with the pumps. What is the density value at each of the pumps? Pinpoint the possible source of the infection.

*Use the projected Cartesian coordinate system for this exercise.* 

## Exercise 7

Create a GeoJSON object from the Voronoi cells, put the cells on the map, colored according to the number of deaths (like on a heatmap). Display the results in the notebook.

*[GeoJSON](http://geojson.org/) is a common format that is used in web applications. It is basically a simple key-value dictionary with a predefined structure for storing geographic information. Once loaded into Python, it behaves like a normal Python dictionary, thus creating or modifying it is quite straightforward. When reading or writing to a file, the `json` module is handy. The advice is to write a function that converts a list of coordinates from a polygon into a GeoJSON feature, then add it to your GeoJSON structure.*

## Exercise 8
(optional)
Download the street network of the area of the deaths from OpenStreetMap using its [map API](https://wiki.openstreetmap.org/wiki/API_v0.6#Retrieving_map_data_by_bounding_box:_GET_.2Fapi.2F0.6.2Fmap). Use street segments given back by the API as edges of a street graph. Develop a method based on the cholera dataset to classify these edges into three categories:
* existed at the time of John Snow
* did not exist at the time of John Snow
* cannot decide

*You'll get the id of each node and way that falls within a bounding box with the GET method of the map API. Use the ids to retrieve the shapes with the help of the ways API.*

*The results will be in XML format. Find a good [XML parser](https://docs.python.org/2/library/xml.etree.elementtree.html#module-xml.etree.ElementTree) to navigate the structures.*

## Exercise 9
(optional)

Plot the streets on the map along with the pumps and the deaths colored by the results of the classification.